In [355]:
import pyspark
from pyspark import SparkFiles
from pyspark import SparkContext
from pyspark.sql import Row
from pyspark.sql import SQLContext
from pyspark.sql import DataFrame
from pyspark.sql.types import *
from pyspark.sql import *
import pyspark.sql.functions as f
from functools import reduce
import pandas as pd
import hanzidentifier

ModuleNotFoundError: No module named 'org'

In [3]:
sc.stop()

In [4]:
# Initialize Pyspark
sc = pyspark.SparkContext()
sqlContext = SQLContext(sc)

In [72]:
spark.conf.set("spark.sql.execution.arrow.pyspark.enabled", "true")
spark.conf.set("spark.sql.execution.arrow.enabled","true")

## Import Data

In [5]:
paths = ["/Users/nicole/Desktop/projects/china bots/chinabots/data/tweets1.csv", "/Users/nicole/Desktop/projects/china bots/chinabots/data/tweets2.csv", "/Users/nicole/Desktop/projects/china bots/chinabots/data/tweets3_1.csv", "/Users/nicole/Desktop/projects/china bots/chinabots/data/tweets3_2.csv", "/Users/nicole/Desktop/projects/china bots/chinabots/data/tweets3_3.csv", "/Users/nicole/Desktop/projects/china bots/chinabots/data/users1.csv", "/Users/nicole/Desktop/projects/china bots/chinabots/data/users2.csv", "/Users/nicole/Desktop/projects/china bots/chinabots/data/users3.csv"]
for path in paths:
    sc.addFile(path)

tweets1 = sqlContext.read.csv(SparkFiles.get(paths[0]), header=True, inferSchema=True)
tweets2 = sqlContext.read.csv(SparkFiles.get(paths[1]), header=True, inferSchema=True)
tweets31 = sqlContext.read.csv(SparkFiles.get(paths[2]), header=True, inferSchema=True)
tweets32 = sqlContext.read.csv(SparkFiles.get(paths[3]), header=True, inferSchema=True)
tweets33 = sqlContext.read.csv(SparkFiles.get(paths[4]), header=True, inferSchema=True)

users1 = sqlContext.read.csv(SparkFiles.get(paths[5]), header=True, inferSchema=True)
users2 = sqlContext.read.csv(SparkFiles.get(paths[6]), header=True, inferSchema=True)
users3 = sqlContext.read.csv(SparkFiles.get(paths[7]), header=True, inferSchema=True)

In [6]:
tweets_df = [tweets1, tweets2, tweets31, tweets32, tweets33]
tweets = reduce(DataFrame.unionAll, tweets_df)

users_df = [users1, users2, users3]
users = reduce(DataFrame.unionAll, users_df)

In [7]:
def convertColumn(df, names, newType):
    for name in names: 
        df = df.withColumn(name, df[name].cast(newType))
    return df 

In [264]:
##### Clean dataframes ####
tweets_int_features = ["userid"]
tweets = convertColumn(tweets, tweets_int_features, IntegerType())
# drop duplicate albums
tweets = reduce(DataFrame.drop, ["follower_count", "following_count"], tweets)

users_int_features = ["userid", "follower_count", "following_count"]
users = convertColumn(users, users_int_features, IntegerType())

In [ ]:
# sqlContext.createDataFrame(users.rdd.takeSample(False, 50, seed=None)).show(5)

In [308]:
def find_random_user(users, tweets):
    """
    Takes in two pyspark dataframes, finds random user, returns two python dataframe: one with user info, one with user tweets
    """
    frame = None
    while frame==None:
        random = users.rdd.takeSample(False, 1, seed=None)
        display_name = random[0].__getattr__("user_display_name")
        if display_name == None:
            continue
        else:
            frame = tweets.filter((f.col("user_display_name") == display_name))
            if frame.count() == 0:
                frame = None
    pd_frame = frame.toPandas()
    pd_random = pd.DataFrame(random[0].asDict(), index=[0])
    return pd_random, pd_frame, random[0], frame

In [309]:
pd_user, pd_tweet, spark_user, spark_tweet = find_random_user(users, tweets)

In [295]:
def find_first_chinese_tweet_pd(user, user_metadata):
    """
    Takes in the user and tweets dataframes and returns information on users language changes if they switched to Chinese
    """
    user["tweet_time"] = pd.to_datetime(user["tweet_time"], errors="coerce")
    user["chinese"] = user["tweet_text"].apply(hanzidentifier.has_chinese)
    user = (user.loc[~user["tweet_time"].isnull()])
    chinese_df = (user.groupby("chinese")["tweet_time"]
                  .apply(list)
                  .to_frame()
                  .reset_index())
    falses = chinese_df.loc[chinese_df["chinese"] == False]
    trues = chinese_df.loc[(chinese_df["chinese"] == True)]
    if (falses.shape[0] == 0) or (trues.shape[0] == 0):
        return None
    
    non_chinese = falses["tweet_time"].item()
    non_chinese.sort(reverse=False)
    num_non = (len(non_chinese))
    
    chinese = trues["tweet_time"].item()
    chinese.sort(reverse=False)
    num_chinese = (len(chinese))
    
    account_creation = user_metadata["account_creation_date"].iloc[0]
    
    if non_chinese[0] > chinese[0]:
        return None
    
    return {"first_chinese": chinese[0],  "last_chinese": chinese[-1], "num_chinese": num_chinese, "first_non_chinese": non_chinese[-1], "last_non_chinese": non_chinese[0], "num_non_chinese": num_non}


In [406]:
def find_first_chinese_tweet_spark(user, tweets, user_pd, tweets_pd):
    """
    Takes in spark user and tweets dataframes and returns information on users language changes if they switched to Chinese
    """
#     tweets.groupBy("tweet_language").filter((f.col()))

    tweets_sub = tweets.select("tweet_language", "tweet_time")
    tweets_sub = tweets_sub.withColumn('tweet_time', 
                   f.to_date(f.unix_timestamp(f.col('tweet_time'), 'MM-dd-yyyy HH:mm').cast("timestamp")))
    tweets_sub = tweets_sub.groupBy("tweet_language").agg(f.min("tweet_time").alias("earliest"))
    tweets_pd["tweet_"]
    return tweets_sub.count()
#     return tweets.withColumn("test3", f.expr("to_date(value, format)")).show()


In [407]:
# find_first_chinese_tweet_spark(spark_user, spark_tweet)
find_first_chinese_tweet_spark(spark_user, spark_tweet)

4

In [328]:
# spark_tweet.withColumn("chinese", hanzidentifier.has_chinese(spark_tweet.tweet_text))

In [311]:
spark_tweet.printSchema()

root
 |-- tweetid: string (nullable = true)
 |-- userid: integer (nullable = true)
 |-- user_display_name: string (nullable = true)
 |-- user_screen_name: string (nullable = true)
 |-- user_reported_location: string (nullable = true)
 |-- user_profile_description: string (nullable = true)
 |-- user_profile_url: string (nullable = true)
 |-- account_creation_date: string (nullable = true)
 |-- account_language: string (nullable = true)
 |-- tweet_language: string (nullable = true)
 |-- tweet_text: string (nullable = true)
 |-- tweet_time: string (nullable = true)
 |-- tweet_client_name: string (nullable = true)
 |-- in_reply_to_userid: string (nullable = true)
 |-- in_reply_to_tweetid: string (nullable = true)
 |-- quoted_tweet_tweetid: string (nullable = true)
 |-- is_retweet: string (nullable = true)
 |-- retweet_userid: string (nullable = true)
 |-- retweet_tweetid: string (nullable = true)
 |-- latitude: string (nullable = true)
 |-- longitude: string (nullable = true)
 |-- quote_co

In [300]:
# iterations = 500
# switch_date = []
# i=0
# while i < iterations:
#     user, user_tweets = find_random_user(users, tweets)
#     diction = find_first_chinese_tweet(user_tweets, user)
#     if diction != None:
#         i+=1
#         switch_date.append(diction["first_chinese"])
#         print(diction["first_chinese"])

In [349]:
pd_tweet["tweet_time"]

0      2019-06-25 07:06
1      2019-06-25 07:13
2      2019-07-17 07:08
3                  None
4      2019-07-14 06:44
             ...       
159                None
160                None
161    2019-06-19 06:36
162    2019-05-09 23:41
163    2019-07-08 09:12
Name: tweet_time, Length: 164, dtype: object